Main Body Architecture Consists of 2 parts
- Convolutional Layers - extract useful features
- Fully Connected (FC) Layers  - using features to classify images

Use YoloV10's backbone as the pretrained model for first part of convolutional layer. 
- Choose not to freeeze the weights as part of the fine tuning process (might freeze due to time)
- Utilize our own custom made training function due to added layers

Initalize Pytorch and YoloV10

In [1]:
#Torch
import torch
#verify torch is installed
print(torch.__version__)
import torch.nn as nn 
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
import torch.optim as optim  

#YoloV10
from ultralytics import YoloV10

#might not be used
import torchvision.transforms as transforms

#set random seed 
torch.manual_seed(1)

'\n#Torch\nimport torch\n#verify torch is installed\nprint(torch.__version__)\nimport torch.nn as nn \nimport torch.nn.functional as F \nimport matplotlib.pyplot as plt \nimport torch.optim as optim  \n\n#YoloV10\nfrom ultralytics import YoloV10\n\n#might not be used\nimport torchvision.transforms as transforms\n\n#set random seed \ntorch.manual_seed(1)\n'

Extract data from files

In [ ]:
#extract info from csv file
#classes: used for end of fc channel - num of possible outputs
numCategory = 10 
#Images and their associated labels
#Adjust after input processing is complete
# (image, label) format preferred
data_training = []
data_validation = []

Create Neural Network Data Structure
- Use freezed YoloV10 backbone along with 2 added layers for convolution part
- Own Customized fully connected layers

In [ ]:
class mushRoom(nn.Modulo):
    def __init__(self, numCategory):
        super(mushRoom, self).__init__()
        #yoloV10 backbone
        self.YOLObackbone = YOLOv10.from_pretrained('jameslahm/yolov10n')
        for param in self.yolov10.parameters():
            param.requires_grad = False  #comment out this line for fine tuning
            #right now the backbone just takes default values, not traiend

        self.pool = nn.MaxPool2d(2,2)

        #check this 2048 number. supposed to be size of yolov10's backbone
        #adjust if necessary
        self.conv1 = nn.Conv2d(2048, 1028, 3)
        self.conv2 = nn.Conv2d(1028, 512, 3)

        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, 64)
        #assume 32 is the number of classes for now
        #adjust after
        self.fc3 = nn.Linear(64, 32)

    def forward(self, x):
        #double check if the function name is called backbone
        #could be called backend
        #
        x = self.yolov10.backbone(x)
        x = self.pool(F.relu(self.conv1))
        x = self.pool(F.relu(self.conv2))

        x = x.view(-1, 512)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
import numpy as np

Helper functions 
- evaluate
- show graph

In [ ]:
def evaluate_model():
    pass

Training Function

In [ ]:
#same as before (lab 2)
#in future, add learning rate function that will decrease as epoch goes on
def train(training_data, validation_data, model, batch_size=64, learning_rate=0.01, num_epoch=30):
    torch.manal_seed(1000)
    criterion = nn.BCEWithLogitsLoss
    optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)

    train_err = np.zeros(num_epoch)
